In [1]:
import pandas as pd
from npi.npi import NPI
from npi.utils.utils import longprint
from npi.pecos import PECOS, medicare_program_engagement, medical_school

### 1. Load in NPI data

In [2]:
npi = NPI(entities=1)

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_entity
* Metadata: {'func': 'get_entity', 'args': ['/work/akilby/npi/data/', None], 'kwargs': {}, 'code': {'get_entity': '-code snipped-'}} 
* (identified) Called functions: ['get_entity']
* Cache found - loading from ID 1597857633589616:
* Cache successfully loaded
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



In [3]:
npi.retrieve('expanded_fullnames')
npi.retrieve('removaldate')
npi.retrieve('licenses')
npi.retrieve('credentials')
npi.retrieve('plocstatename')

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_name
* Metadata: {'func': 'get_name', 'args': ['/work/akilby/npi/data/', None, 3759104195531573822, 'pfname'], 'kwargs': {}, 'code': {'get_name': '-code snipped-'}} 
* (identified) Called functions: ['get_name']
* Cache found - loading from ID 1596588593764036:
* Cache successfully loaded
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
* Function: get_name
* Metadata: {'func': 'get_name', 'args': ['/work/akilby/npi/data/', N

### 2. Define functions to compare ME data to NPI data

In [4]:
def conflicting_middle_names_npi(df, identifier, firstname, middlename, lastname, npi):
    """
    Takes a dataframe with first name, middle name, and last name, cleans, and checks
    conflicts with the NPI dataset. These pairings should be subsequently removed from
    any NPI-ME matches
    """
    middlenames_all = df[[identifier, firstname, middlename, lastname]].fillna('').assign(**{middlename: lambda df: df[middlename].str.replace('.', '').str.strip()})
    middlenames_init = middlenames_all.query(f'{middlename} > ""').loc[lambda df: df[middlename].str.len() == 1]
    middlenames_full = middlenames_all.query(f'{middlename} > ""').loc[lambda df: df[middlename].str.len() > 1]
    return pd.concat([middlenames_full.merge(npi.loc[npi.pmname.str.len()>1], right_on=['pfname', 'plname'], left_on=[firstname, lastname]).query(f'{middlename}!=pmname')[[identifier, 'npi']].drop_duplicates(),
                      middlenames_init.merge(npi.loc[npi.pmname.str.len()==1], right_on=['pfname', 'plname'], left_on=[firstname, lastname]).query(f'{middlename}!=pmname')[[identifier, 'npi']].drop_duplicates()], ignore_index=True).drop_duplicates()


def match_to_npi_with_names(df, identifier, firstname, middlename, lastname, npi):
    """
    Matches on first name-last name, while purging any matches with mismatching middle names
    """
    df = df.assign(**{firstname: lambda d: d[firstname].str.upper().str.strip(),
                      middlename: lambda d: d[middlename].str.upper().str.strip(),
                      lastname: lambda d: d[lastname].str.upper().str.strip()})
    conflicts = conflicting_middle_names_npi(
        df, identifier, firstname, middlename, lastname, npi)
    return (df
            .merge(npi, right_on=['pfname', 'plname'], left_on=[firstname, lastname])
            .merge(conflicts, how='left', indicator=True)
            .query('_merge!="both"')[[identifier, 'npi']]
            .drop_duplicates()
            .sort_values([identifier, 'npi'])
            .reset_index(drop=True))


def death_and_deactivation_dates(df, deathdate):
    """
    Must already have npideactdate in frame, specify death date var
    """
    return (df
            .query(f'{deathdate}<=npideactdate')
            .assign(**{'days_betw_death_deact': 
                       lambda df: df.npideactdate - df[deathdate]}))


def retrieve_supplementary_info(df, npi):
    """
    Make wide data, unique at NPI, for categories, credentials, and licensing
    """
    states = npi.plocstatename.merge(df['npi'].drop_duplicates()).groupby('npi').last().drop(columns='month')
    assert states.index.is_unique

    cats = npi.credentials[['npi', 'cat']].merge(df['npi'].drop_duplicates()).drop_duplicates().sort_values(['npi', 'cat']).assign(num=lambda df: df.groupby('npi').cumcount()+1).set_index(['npi', 'num']).unstack().dropna(1, how='all').dropna(how='all')
    new_column_names = [str(x[0])+str(x[1]) for x in cats.columns.tolist()]
    cats.columns = new_column_names
    assert cats.index.is_unique

    creds = npi.credentials[['npi', 'pcredential']].merge(df['npi'].drop_duplicates()).drop_duplicates().sort_values(['npi', 'pcredential']).assign(num=lambda df: df.groupby('npi').cumcount()+1).set_index(['npi', 'num']).unstack().dropna(1, how='all').dropna(how='all')
    new_column_names = [str(x[0])+str(x[1]) for x in creds.columns.tolist()]
    creds.columns = new_column_names
    assert creds.index.is_unique

    lics = npi.licenses[['npi', 'PLICSTATE', 'PLICNUM']].merge(df['npi'].drop_duplicates()).drop_duplicates().sort_values(['npi', 'PLICSTATE', 'PLICNUM']).assign(num=lambda df: df.groupby('npi').cumcount()+1).set_index(['npi', 'num']).unstack().dropna(1, how='all')
    new_column_names = [str(x[0])+str(x[1]) for x in lics.columns.tolist()]
    lics.columns = new_column_names
    lics = lics[sorted([x for x in lics.columns], key=lambda s: s[-1:])]
    assert lics.index.is_unique

    return df.merge(lics.reset_index(), how='left').merge(cats.reset_index(), how='left').merge(creds.reset_index(), how='left').merge(states.reset_index(), how='left')


def identify_matching_state(df, target_col_name):
    """
    Takes DF with a target state column and checks if it matches any other column in the dataframe
    """
    df_compare_cols = [x for x in df.columns if x!=target_col_name]
    return pd.concat([df[target_col_name]==df[col] for col in df_compare_cols], 1).any(1).rename('has_matching_state')


### 3. Cook County

In [5]:
df = pd.read_excel('/work/akilby/state_mortality/cookCountyILMErecordsreceivedJune15.xlsx')

In [6]:
# Specific cleaning to Cook county - weird spaces in state name
df = df.assign(**{'DEATH_STATE': lambda d: d['DEATH_STATE'].fillna('IL').str.strip()})

In [7]:
# Set relevant column names and other objects

deathdate = 'DEATH_DATE'
deathstate = 'DEATH_STATE'
firstname = 'DECEDENT_FIRST_NAME'
middlename = 'DECEDENT_MIDDLE_NAME'
lastname = 'DECEDENT_LAST_NAME'
identifier = 'CASENUMBER'

statename = 'IL'
placename = 'cook_county'

In [8]:
npi_name_matches = match_to_npi_with_names(df, identifier, firstname, middlename, lastname, npi.expanded_fullnames)
merged = df.merge(npi_name_matches).merge(npi.removaldate)

merged = death_and_deactivation_dates(merged, deathdate)
merged = retrieve_supplementary_info(merged, npi)
merged = pd.concat([merged, identify_matching_state(merged[[deathstate, 'plocstatename'] + [x for x in merged.columns if 'LICSTATE' in x]], deathstate)], axis=1)

sort_first = [identifier, deathdate,  'npi', 'npideactdate', 'has_matching_state', 'days_betw_death_deact']
final = merged.sort_values(['has_matching_state', 'days_betw_death_deact']).reset_index(drop=True)[sort_first + [x for x in merged.columns if x not in sort_first]]
final.to_csv('/work/akilby/state_mortality/%s.csv' % placename)    

final

,CASENUMBER,DEATH_DATE,npi,npideactdate,has_matching_state,days_betw_death_deact,DECEDENT_FIRST_NAME,DECEDENT_MIDDLE_NAME,DECEDENT_LAST_NAME,RESIDENCE_ZIP,...,PLICSTATE6,PLICNUM6,PLICSTATE7,PLICNUM7,cat1,pcredential1,pcredential2,pcredential3,pcredential4,plocstatename
0,ME2020-05650,2020-05-15 10:30:00,1376690578,2020-05-18,False,2 days 13:30:00,BARRY,NaN,SUGARMAN,60645,...,NaN,NaN,NaN,NaN,MD/DO,MD,NaN,NaN,NaN,MA
1,ME2020-02821,2020-04-04 17:05:00,1609808468,2020-04-08,False,3 days 06:55:00,JOSE,NaN,GARCIA,60164,...,NaN,NaN,NaN,NaN,PA,PA-C,NaN,NaN,NaN,CA
2,ME2020-05232,2020-04-28 09:45:00,1104962836,2020-05-04,False,5 days 14:15:00,JOHN,NaN,WILLIAMS,60644,...,NaN,NaN,NaN,NaN,MD/DO,MD,NaN,NaN,NaN,CA
3,ME2020-03565,2020-04-23 21:32:00,1255302543,2020-05-04,False,10 days 02:28:00,CATHERINE,NaN,MYERS,60025,...,NaN,NaN,NaN,NaN,NaN,WHCNP,NaN,NaN,NaN,MN
4,ME2020-04973,2020-05-08 02:30:00,1053377465,2020-05-20,False,11 days 21:30:00,LORRAINE,NaN,HARRIS,60649,...,NaN,NaN,NaN,NaN,NaN,LAC,NaN,NaN,NaN,NC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61,ME2020-04632,2020-05-05 03:18:00,1780817775,2020-06-30,True,55 days 20:42:00,ROBERT,NaN,WORWA,60157,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IL
62,ME2020-04251,2020-05-01 11:15:00,1932614088,2020-06-30,True,59 days 12:45:00,YOLANDA,NaN,DANIELS,60130,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IL
63,ME2020-04318,2020-05-02 01:47:00,1609814482,2020-08-03,True,92 days 22:13:00,ROBERT,NaN,SIMMONS,60636,...,NaN,NaN,NaN,NaN,Dentist,DDS,NaN,NaN,NaN,IL
64,ME2020-02884,2020-04-16 08:20:00,1720180516,2020-07-20,True,94 days 15:40:00,DALE,NaN,ROSENFIELD,60026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IL


### 4. Broward County

In [9]:
df = pd.read_csv('/work/akilby/state_mortality/Broward COVID-19 Deaths 2020-06-08.csv')

In [10]:
# Cleaning specific to Broward County
df = df.rename(columns = {'First-Name': 'FirstName', 'Last-Name': 'LastName', 'Unnamed: 4': 'MiddleName'})
df = df.assign(DeathState='FL', 
               DeathDate=pd.to_datetime(df['DeathDate']))

In [11]:
# Set relevant column names and other objects
deathdate = 'DeathDate'
deathstate = 'DeathState'
firstname = 'FirstName'
middlename = 'MiddleName'
lastname = 'LastName'
identifier = 'CaseNum'

statename = 'FL'
placename = 'broward_county'

In [12]:
# Procedure to identify possible matches and save to disk
npi_name_matches = match_to_npi_with_names(df, identifier, firstname, middlename, lastname, npi.expanded_fullnames)
merged = df.merge(npi_name_matches).merge(npi.removaldate)
merged = death_and_deactivation_dates(merged, deathdate)

merged = retrieve_supplementary_info(merged, npi)
merged = pd.concat([merged, identify_matching_state(merged[[deathstate, 'plocstatename'] + [x for x in merged.columns if 'LICSTATE' in x]], deathstate)], axis=1)

sort_first = [identifier, deathdate,  'npi', 'npideactdate', 'has_matching_state', 'days_betw_death_deact']
final = merged.sort_values(['has_matching_state', 'days_betw_death_deact']).reset_index(drop=True)[sort_first + [x for x in merged.columns if x not in sort_first]]
final.to_csv('/work/akilby/state_mortality/%s.csv' % placename)    

final

,CaseNum,DeathDate,npi,npideactdate,has_matching_state,days_betw_death_deact,CaseType,LastName,FirstName,MiddleName,...,deathzip,DeathState,PLICSTATE1,PLICNUM1,PLICSTATE2,PLICNUM2,cat1,pcredential1,pcredential2,plocstatename
0,BME2020-1554,2020-04-16,1932586369,2020-07-10,False,85 days,ME,Smith,Stephanie,NaN,...,NaN,FL,TX,1110583,NaN,NaN,NaN,NaN,NaN,TX
1,BME2020-1275,2020-04-03,1417048687,2020-07-17,False,105 days,ME,Lee,Mary,NaN,...,NaN,FL,TX,F8204,NaN,NaN,MD/DO,DO,NaN,TX
2,BME2020-2270,2020-05-22,1245540186,2020-06-08,True,17 days,ME,Omobasuyi,David,NaN,...,NaN,FL,FL,2870242,NaN,NaN,NP,ARNP,DHSC,FL
3,BME2020-1046,2020-03-24,1154462059,2020-04-14,True,21 days,ME,Hsu,Alex,NaN,...,NaN,FL,FL,ME0045523,NaN,NaN,MD/DO,MD,NaN,FL
4,BME2020-1175,2020-03-31,1356638381,2020-05-18,True,48 days,ME,Espinoza,Orlando,NaN,...,NaN,FL,FL,ME111407,PR,28213,MD/DO,MD,NaN,FL
